In [2]:
import torch
import torch.nn as nn
from torch import tensor
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
from IPython import display
import pylab as pl
from src.data_structures import Instance
import seaborn as sns
torch.set_default_tensor_type(torch.DoubleTensor)


In [3]:
instance = Instance.generate(256,1)
print(len(instance.polynomial_gains.keys()))

1464


In [4]:
#Primero explorar que tan factible es resolver esto para las polys de 2 dimensiones
polys = torch.zeros((instance.n_items,instance.n_items))

#Hago la matriz simetrica :D
treshold = 0.8
func = np.random.random
for i in range(len(polys)):
    for j in range(len(polys)):
        if np.random.random() > treshold:
            polys[i,j] = func()
            polys[j,i] = polys[j,i]
            if i == j:
                polys[i,j] = 0

sns.heatmap(polys)
plt.legend()
plt.savefig("real.png")
plt.clf()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Figure size 640x480 with 0 Axes>

In [5]:
from itertools import combinations
#Genero los inputs
inputs = list(combinations(range(instance.n_items),2))
def int_to_vec(tupla):
    output = np.zeros(instance.n_items)
    for x in tupla:
        output[x]= 1
    return output
values = []
inputs2 = []
for tupla in inputs:
    if polys[tupla[0],tupla[1]] != 0:
        values.append(polys[tupla[0],tupla[1]])
        inputs2.append(tupla)

values = tensor(values)
tuplas = inputs2.copy()
inputs = np.array(list(map(int_to_vec,inputs2)))


In [6]:
entradas = instance.n_items
salidas = 1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden = 10
        # Define the layers of your neural network
        self.fc1 = nn.Linear(entradas, hidden)
        self.fc2 = nn.Linear(hidden, salidas)
        self.activation = nn.LeakyReLU()
        self.output_activation = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.output_activation(self.fc1(x))
        x = self.output_activation(self.fc2(x))
        return x

net = Net()
criterion = nn.L1Loss()
learning_rate = 1
optimizer = torch.optim.ASGD(net.parameters(), lr=learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,0.1,5e-3,10000)
train_in = inputs
expected_out = values
epoch = 0
train_tensor = torch.tensor(train_in)

In [51]:
for epoch in range(10):

    for i,value in enumerate(expected_out):
        salida_predicha = net(train_tensor[i])[0]
        a = salida_predicha
        #a.requires_grad = True
        #b.requires_grad = True
        loss = criterion(a,value)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #scheduler.step()
    sys.stdout.write(f"\rEntrenando :D {epoch+1} {loss.item()} {optimizer.param_groups[0]['lr']}")
    sys.stdout.flush()

        #a = net(train_tensor).T
        #predicted = np.zeros((instance.n_items,instance.n_items))
        #for i,tupla in enumerate(tuplas):
        #    predicted[tupla[0],tupla[1]] = a[0][i]
        #sns.heatmap(predicted)
        #plt.savefig("predicted.png")
        #plt.clf()
        #sns.heatmap(np.abs(polys-predicted))
        #plt.savefig("difference.png")
        #plt.clf()


KeyboardInterrupt: 

In [ ]:
a = net(tensor(train_in)).T


In [ ]:
import seaborn as sns

predicted = np.zeros((instance.n_items,instance.n_items))
for i,tupla in enumerate(tuplas):
    predicted[tupla[0],tupla[1]] = a[0][i]

sns.heatmap(polys)
plt.legend()
plt.savefig("real.png")
plt.clf()
sns.heatmap(predicted)
plt.savefig("predicted.png")
plt.clf()
plt.title(loss.item())
sns.heatmap(np.abs(polys-predicted))
plt.savefig("difference.png")
plt.clf()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Figure size 640x480 with 0 Axes>

In [ ]:
test = np.zeros(instance.n_items)
a = net(tensor([test]))
a


tensor([[0.6675]], grad_fn=<SigmoidBackward0>)